In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# set pandas to display all columns
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('new_222_data_imputed.csv')

# drop the county column
df = df.drop(['county'], axis=1)
df.head()

,prev_year_access_to_exercise_opportunities,prev_year_adult_obesity,prev_year_adult_smoking,prev_year_air_pollution_particulate_matter,prev_year_alcohol_impaired_driving_deaths,prev_year_children_in_poverty,prev_year_children_in_single_parent_households,prev_year_dentists,prev_year_diabetes_monitoring,prev_year_drinking_water_violations,prev_year_driving_alone_to_work,prev_year_excessive_drinking,prev_year_flu_vaccinations,prev_year_food_environment_index,prev_year_high_school_completion,prev_year_income_inequality,prev_year_injury_deaths,prev_year_long_commute_driving_alone,prev_year_low_birthweight,prev_year_mammography_screening,prev_year_mental_health_providers,prev_year_physical_inactivity,prev_year_poor_mental_health_days,prev_year_poor_or_fair_health,prev_year_poor_physical_health_days,prev_year_premature_death,prev_year_preventable_hospital_stays,prev_year_primary_care_physicians,prev_year_severe_housing_problems,prev_year_sexually_transmitted_infections,prev_year_social_associations,prev_year_some_college,prev_year_teen_births,prev_year_unemployment,prev_year_uninsured,prev_year_violent_crime,prev_yr_rent_50_0,prev_yr_rent_50_1,prev_yr_rent_50_2,prev_yr_rent_50_3,prev_yr_rent_50_4,coastline_flag,Top_Marginal_Personal_Income_Tax_Rate,Top_Marginal_Corporate_Income_Tax_Rate,"Personal_Income_Tax_Progressivity_(change in tax liability per $1,000 of income)","Property_Tax_Burden_(per $1,000 of personal income)","Sales_Tax_Burden(per $1,000 of personal income)","Remaining_Tax_Burden(per $1,000 of personal income)",Estate_Inheritance_Tax_Levied_Flag,Recently_Legislated_Tax_Changes,Debt_Service_as_a_Share_of_Tax_Revenue,"Public_Employees_Per_10,000_of_Population_(full-time equivalent)","State_Liability_System_Survey_(tort litigation treatment, judicial impartiality, etc.)",State_Minimum_Wage_(federal floor is $7.25),Average Workers_Compensation_Costs_(per $100 of payroll),Right-to-Work_State_Flag_(option to join or support a union),Tax_Expenditure_Limits,tax,loan,business,rent,college,apartments,office,insurance,Business plan templates,Small business ideas,How to start a small business,office space for rent,website hosting service,freelancing,side hustle,ecommerce,tax_pct,loan_pct,business_pct,rent_pct,college_pct,apartments_pct,office_pct,insurance_pct,Business plan templates_pct,Small business ideas_pct,How to start a small business_pct,office space for rent_pct,website hosting service_pct,freelancing_pct,side hustle_pct,ecommerce_pct,deaths_number,death_pct_diff,GPT_Autoencoder_dim0,GPT_Autoencoder_dim1,GPT_Autoencoder_dim2,GPT_Autoencoder_dim3,GPT_Autoencoder_dim4,GPT_Autoencoder_dim5,GPT_Autoencoder_dim6,GPT_Autoencoder_dim7,GPT_Autoencoder_dim8,GPT_Autoencoder_dim9,GPT_PCA_dim0,GPT_PCA_dim1,GPT_PCA_dim2,GPT_PCA_dim3,GPT_PCA_dim4,GPT_PCA_dim5,GPT_PCA_dim6,GPT_PCA_dim7,GPT_PCA_dim8,GPT_PCA_dim9,GPT_OH_Connected_Economy,GPT_OH_Government_Support,GPT_OH_Large_Customer_Base,GPT_OH_Low_Cost_of_Living,GPT_OH_Natural_Resources,GPT_OH_Supportive_Community,GPT_OH_Limited_Financing,GPT_OH_Limited_Tech_n_Infra,GPT_OH_Low_Pop_Bad_Weather,GPT_OH_Tough_Labor_Market,GPT_OH_Unfavorable_Location,GPT_OH_Various_Downsides,National_PI,pop_2010,pop_2015,pop_2020,pop_5yr_pct_chg,pop_10yr_pct_chg,PrevYr_RGDP,Prev_Yr_Agri_Pct_GDP,Prev_Yr_Mine_Pct_GDP,Prev_Yr_Retl_Pct_GDP,Prev_Yr_Manu_Pct_GDP,Prev_Yr_Info_Pct_GDP,Prev_Yr_FinS_Pct_GDP,Prev_Yr_EduS_Pct_GDP,Prev_Yr_EntS_Pct_GDP,Prev_Yr_Gvmt_Pct_GDP,Prev_Yr_GoMn_Pct_GDP,Prev_Yr_GoTr_Pct_GDP,Prev_Yr_GoMf_Pct_GDP,two_yrs_prior_pct_bb,two_yrs_prior_pct_college,two_yrs_prior_pct_foreign_born,two_yrs_prior_pct_it_workers,two_yrs_prior_median_hh_inc,microbusiness_density
0,0.69,0.33,0.20,10.4,0.33,0.16,0.25,2940.0,0.84,0.0,0.86,0.14,0.42,6.7,0.89,5.1,68.0,0.38,0.09,0.39,3490.0,0.31,4.9,0.20,4.5,7800.0,6650.0,2140.0,0.14,504.5,12.2,0.60,24.0,0.027,0.10,272.0,685.0,820.0,972.0,1229.0,1628.0,0.0,0.04,0.06,-1.8,14.37,26.04,20.87,0.0,0.55,0.06,583.5,65.6,7.25,1.33,1.0,0.0,25.9125,15.9625,21.779167,25.095833,52.

In [3]:
# create a copy of the dataframe
df_copy = df.copy()

X = df_copy.iloc[:, :-1]
y = df_copy.iloc[:, -1]

# normalize the data
X_norm = X.copy()
X_norm = X_norm.loc[:, X.std() != 0]
X_norm = (X_norm - X_norm.mean()) / X_norm.std()


In [4]:
from sklearn.feature_selection import SequentialFeatureSelector
N_features_sel = 20

In [5]:
# forward selection with linear regression
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()
sfs = SequentialFeatureSelector(lr_model, n_features_to_select=N_features_sel, direction='forward', n_jobs=-1)
sfs.fit(X_norm, y)
lr_forward_importants = sfs.get_feature_names_out()
print('linear regression forward selection: ')
print(lr_forward_importants)

linear regression forward selection: 
['prev_year_adult_obesity' 'prev_year_adult_smoking'
 'prev_year_high_school_completion' 'prev_year_long_commute_driving_alone'
 'prev_year_mammography_screening' 'prev_year_severe_housing_problems'
 'prev_year_some_college' 'prev_yr_rent_50_4'
 'Property_Tax_Burden_(per $1,000 of personal income)'
 'Remaining_Tax_Burden(per $1,000 of personal income)' 'pop_2010'
 'pop_5yr_pct_chg' 'PrevYr_RGDP' 'Prev_Yr_Agri_Pct_GDP'
 'Prev_Yr_Mine_Pct_GDP' 'Prev_Yr_FinS_Pct_GDP' 'Prev_Yr_EntS_Pct_GDP'
 'Prev_Yr_Gvmt_Pct_GDP' 'two_yrs_prior_pct_college'
 'two_yrs_prior_pct_foreign_born']


In [6]:
X_lr = X_norm[lr_forward_importants]
y_lr = y 

# use statsmodels to get the p-values
import statsmodels.api as sm
X_lr = sm.add_constant(X_lr)
lr_model = sm.OLS(y_lr, X_lr).fit()
# rank the features by their p-values
lr_p_values = lr_model.pvalues
lr_p_values = lr_p_values.sort_values(ascending=True)

# sorted features by p-values as a list
lr_sorted = list(lr_p_values.index)
lr_p_values

const                                                  0.000000e+00
two_yrs_prior_pct_college                              5.360497e-12
Prev_Yr_FinS_Pct_GDP                                   9.352297e-09
Remaining_Tax_Burden(per $1,000 of personal income)    5.404674e-08
pop_5yr_pct_chg                                        2.017696e-06
PrevYr_RGDP                                            4.187878e-06
prev_year_mammography_screening                        1.233045e-04
prev_year_adult_obesity                                2.154562e-04
prev_year_high_school_completion                       6.416157e-04
Prev_Yr_Agri_Pct_GDP                                   6.754987e-04
prev_year_long_commute_driving_alone                   8.980575e-04
prev_year_adult_smoking                                9.988696e-04
prev_year_severe_housing_problems                      1.069120e-03
Property_Tax_Burden_(per $1,000 of personal income)    1.131559e-03
Prev_Yr_Gvmt_Pct_GDP                            

In [7]:
# forward selection with KNN regression
from sklearn.neighbors import KNeighborsRegressor

knn_model = KNeighborsRegressor()
sfs = SequentialFeatureSelector(knn_model, n_features_to_select=N_features_sel, direction='forward', n_jobs=-1)

sfs.fit(X_norm, y)
knn_forward_importants = sfs.get_feature_names_out()
print('KNN forward selection: ')
print(knn_forward_importants)

KNN forward selection: 
['prev_year_access_to_exercise_opportunities' 'prev_year_adult_smoking'
 'prev_year_poor_or_fair_health'
 'Public_Employees_Per_10,000_of_Population_(full-time equivalent)'
 'State_Liability_System_Survey_(tort litigation treatment, judicial impartiality, etc.)'
 'office space for rent' 'website hosting service' 'side hustle'
 'office_pct' 'website hosting service_pct' 'side hustle_pct'
 'GPT_OH_Low_Cost_of_Living' 'GPT_OH_Tough_Labor_Market' 'National_PI'
 'pop_2020' 'PrevYr_RGDP' 'Prev_Yr_Mine_Pct_GDP' 'Prev_Yr_Retl_Pct_GDP'
 'Prev_Yr_GoMn_Pct_GDP' 'two_yrs_prior_pct_college']


In [8]:
# forward selection with DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor

dt_model = DecisionTreeRegressor()
sfs = SequentialFeatureSelector(dt_model, n_features_to_select=N_features_sel, direction='forward', n_jobs=-1)
sfs.fit(X_norm, y)
dt_forward_importants = sfs.get_feature_names_out()
print('DecisionTree forward selection: ')
print(dt_forward_importants)

DecisionTree forward selection: 
['prev_year_adult_smoking' 'prev_year_drinking_water_violations'
 'prev_yr_rent_50_2' 'coastline_flag' 'Tax_Expenditure_Limits'
 'Business plan templates' 'website hosting service'
 'website hosting service_pct' 'side hustle_pct' 'death_pct_diff'
 'GPT_OH_Connected_Economy' 'GPT_OH_Low_Cost_of_Living'
 'GPT_OH_Natural_Resources' 'GPT_OH_Supportive_Community'
 'GPT_OH_Limited_Financing' 'GPT_OH_Low_Pop_Bad_Weather'
 'GPT_OH_Tough_Labor_Market' 'GPT_OH_Unfavorable_Location' 'National_PI'
 'pop_2015']


In [9]:
X_dt = X_norm[dt_forward_importants]
y_lt = y

# fit a decision tree model and get the feature importances
dt_model = DecisionTreeRegressor()
dt_model.fit(X_dt, y_lt)
dt_importances = dt_model.feature_importances_
dt_importances = pd.Series(dt_importances, index=X_dt.columns)
dt_importances = dt_importances.sort_values(ascending=False)
dt_sorted = list(dt_importances.index)
dt_importances

prev_year_adult_smoking                0.205041
pop_2015                               0.194172
prev_yr_rent_50_2                      0.185307
death_pct_diff                         0.124552
GPT_OH_Tough_Labor_Market              0.114702
side hustle_pct                        0.072812
Tax_Expenditure_Limits                 0.052164
GPT_OH_Natural_Resources               0.012835
Business plan templates                0.007933
GPT_OH_Limited_Financing               0.007144
website hosting service                0.005459
GPT_OH_Supportive_Community            0.003685
GPT_OH_Low_Pop_Bad_Weather             0.003058
GPT_OH_Unfavorable_Location            0.002954
GPT_OH_Low_Cost_of_Living              0.002799
prev_year_drinking_water_violations    0.002534
website hosting service_pct            0.001106
coastline_flag                         0.000921
GPT_OH_Connected_Economy               0.000821
National_PI                            0.000000
dtype: float64

In [10]:
# forward selection with Random Forest
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(max_depth=3, n_estimators=30)
sfs = SequentialFeatureSelector(rf_model, n_features_to_select=N_features_sel, direction='forward', n_jobs=-1)
sfs.fit(X_norm, y)
rf_forward_importants = sfs.get_feature_names_out()
print('random forest forward selection: ')
print(rf_forward_importants)

random forest forward selection: 
['prev_year_access_to_exercise_opportunities'
 'prev_year_alcohol_impaired_driving_deaths'
 'prev_year_driving_alone_to_work' 'prev_year_low_birthweight'
 'prev_year_uninsured' 'prev_yr_rent_50_1'
 'Right-to-Work_State_Flag_(option to join or support a union)'
 'Business plan templates' 'website hosting service' 'business_pct'
 'How to start a small business_pct' 'GPT_OH_Connected_Economy'
 'GPT_OH_Government_Support' 'GPT_OH_Large_Customer_Base'
 'GPT_OH_Limited_Tech_n_Infra' 'PrevYr_RGDP' 'Prev_Yr_Mine_Pct_GDP'
 'Prev_Yr_FinS_Pct_GDP' 'Prev_Yr_EntS_Pct_GDP' 'two_yrs_prior_pct_college']


In [11]:
# fit a random forest model and get the feature importances
X_rf = X_norm[rf_forward_importants]
y_rf = y
rf_model = RandomForestRegressor(max_depth=3, n_estimators=30)
rf_model.fit(X_rf, y_rf)
rf_importances = rf_model.feature_importances_
rf_importances = pd.Series(rf_importances, index=X_rf.columns)
rf_importances = rf_importances.sort_values(ascending=False)
rf_sorted = list(rf_importances.index)
rf_importances

two_yrs_prior_pct_college                                       0.622705
prev_yr_rent_50_1                                               0.127251
business_pct                                                    0.099482
prev_year_access_to_exercise_opportunities                      0.065470
Prev_Yr_FinS_Pct_GDP                                            0.032701
Prev_Yr_EntS_Pct_GDP                                            0.014746
prev_year_alcohol_impaired_driving_deaths                       0.008960
GPT_OH_Connected_Economy                                        0.007784
PrevYr_RGDP                                                     0.007475
prev_year_low_birthweight                                       0.006034
How to start a small business_pct                               0.005829
prev_year_driving_alone_to_work                                 0.001563
prev_year_uninsured                                             0.000000
Right-to-Work_State_Flag_(option to join or support

In [12]:
# forward selection with BayesianRidge
from sklearn.linear_model import BayesianRidge

br_model = BayesianRidge()
sfs = SequentialFeatureSelector(br_model, n_features_to_select=N_features_sel, direction='forward', n_jobs=-1)
sfs.fit(X_norm, y)
br_forward_importants = sfs.get_feature_names_out()
print('BayesianRidge forward selection: ')
print(br_forward_importants)

BayesianRidge forward selection: 
['prev_year_adult_obesity' 'prev_year_adult_smoking'
 'prev_year_children_in_single_parent_households'
 'prev_year_high_school_completion' 'prev_year_long_commute_driving_alone'
 'prev_year_mammography_screening' 'prev_year_severe_housing_problems'
 'prev_year_some_college' 'prev_yr_rent_50_4'
 'Property_Tax_Burden_(per $1,000 of personal income)'
 'Remaining_Tax_Burden(per $1,000 of personal income)' 'pop_5yr_pct_chg'
 'PrevYr_RGDP' 'Prev_Yr_Agri_Pct_GDP' 'Prev_Yr_Mine_Pct_GDP'
 'Prev_Yr_FinS_Pct_GDP' 'Prev_Yr_EntS_Pct_GDP' 'Prev_Yr_Gvmt_Pct_GDP'
 'two_yrs_prior_pct_college' 'two_yrs_prior_pct_foreign_born']


In [13]:
X_br = X_norm[br_forward_importants]
y_br = y

br_model = BayesianRidge()
br_model.fit(X_br, y_br)
br_importances = br_model.coef_
br_importances = pd.Series(br_importances, index=X_br.columns)
br_importances = br_importances.sort_values(ascending=False)
br_sorted = list(br_importances.index)

In [14]:
# put all the forward selection results into a dataframe
forward_selection_df = pd.DataFrame({'OLS': lr_sorted[:-1],
                                        'KNN': knn_forward_importants,
                                        'DecisionTree': dt_sorted,
                                        'RandomForest': rf_sorted,
                                        'BayesianRidge': br_sorted})
forward_selection_df

,OLS,KNN,DecisionTree,RandomForest,BayesianRidge
0,const,prev_year_access_to_exercise_opportunities,prev_year_adult_smoking,two_yrs_prior_pct_college,two_yrs_prior_pct_college
1,two_yrs_prior_pct_college,prev_year_adult_smoking,pop_2015,prev_yr_rent_50_1,Prev_Yr_FinS_Pct_GDP
2,Prev_Yr_FinS_Pct_GDP,prev_year_poor_or_fair_health,prev_yr_rent_50_2,business_pct,pop_5yr_pct_chg
3,"Remaining_Tax_Burden(per $1,000 of personal in...","Public_Employees_Per_10,000_of_Population_(ful...",death_pct_diff,prev_year_access_to_exercise_opportunities,prev_year_high_school_completion
4,pop_5yr_pct_chg,State_Liability_System_Survey_(tort litigation...,GPT_OH_Tough_Labor_Market,Prev_Yr_FinS_Pct_GDP,PrevYr_RGDP
5,PrevYr_RGDP,office space for rent,side hustle_pct,Prev_Yr_EntS_Pct_GDP,"Remaining_Tax_Burden(per $1,000 of personal in..."
6,prev_year_mammography_screening,website hosting service,Tax_Expenditure_Limits,prev_year_alcohol_impaired_driving_deaths,Prev_Yr_EntS_Pct_GDP
7,prev_year_adult_obesity,side hustle,GPT_OH_Natural_Resources,GPT_OH_Connected_Economy,two_yrs_prior_pct_foreign_born
8,prev_year_high_school_completion,office_pct,Business plan templates,PrevYr_RGDP,prev_year_severe_housing_problems
9,Prev_Yr_Agri_Pct_GDP,website hosting service_pct,GPT_OH_Limited_Financing,prev_year_low_birthweight,prev_yr_rent_50_4


In [16]:
# save the forward selection results to a csv file
forward_selection_df.to_csv('forward_selection_result_corr.csv', index=False)